In [9]:
import requests
import time
import pandas as pd
import backoff
import missingno as sns

## **CALCULOS PARA LA BUSQUEDA DE PUNTOS**
A continacion se generara el codigo para obtener los datos de locales que vendan cortinas en chile la metodologia se encuentra en el directorio con el nombre de `Api Google (Tiendas de Cortinas)` dado que se trata de la iteracion en un espacio geografico de un pais se realizara una iteracion de cuadriculas predefinidas. estas cuadriculas estan pensadas para poder extraer datos de manera general pero de manera tal que no perdamos tantos datos. 

La operacion matematica que  se realiza en el codigo es calcular el tamaño del "paso" necesario para iterar en un radio de 30000 metros una busqueda en una cuadricula de busqueda en latitud (`lat_step`) y longitud (`lng_step`) dentro de los límites geograficos decididos.

### **EXPLICACION:**

1. **Para la latitud (`lat_step`):**
   - `lat_max - lat_min`: Determino la distancia total en grados de latitud entre el limite maximo y mínimo.
   - `/ num_points_lat`: Divido esta distancia total por el numero de puntos deseados en la cuadricula de latitud.
   - Resultado: `lat_step` representa la distancia en grados de latitud entre cada punto consecutivo en la cuadricula generada.

2. **Para la longitud (`lng_step`):**
   - `lng_max - lng_min`: Determino la distancia total en grados de longitud entre el limite maximo y mínimo.
   - `/ num_points_lng`: Divide esta distancia total por el número de puntos deseados en la cuadrícula de longitud.
   - Resultado: `lng_step` representa la distancia en grados de longitud entre cada punto consecutivo en la cuadricula generada.

### **EJEMPLO:**

Supongamos que tenemos los siguientes valores:

- `lat_min = -56.0`
- `lat_max = -17.5`
- `num_points_lat = 20`


la fórmula para calcular `lat_step` seria:

$$
\text{lat\_step} = \frac{-17.5 - (-56.0)}{20} = \frac{38.5}{20} = 1.925 =  213,975 metros (aproximadamente)
$$

Esto significa que `lat_step` es igual a 1.925 grados de latitud, lo que equivale aproximadamente a unos 213 metros. Por lo tanto, cada punto en la cuadrícula de latitud estara separado por aproximadamente 213 metros. Esto significa que la busqueda se realiza en puntos específicos de esta cuadricula y no se consideran las ubicaciones intermedias.

In [5]:
# clave de API
api_key = ''

# Defino una cantidad de puntos en la cuadrícula (a mayor puntos mas mas desagregada la busqueda)

num_points_lat = 20  
num_points_lng = 20  

# Defino los límites geograficos de Chile
lat_min = -56.0  
lat_max = -17.5  
lng_min = -75.0  
lng_max = -66.0  

# Defino un radio de búsqueda en metros
radius = 30000  

# palabra clave de la busqueda
keyword = 'Tiendas de Cortinas'

# funcion para realizar una busqueda 
@backoff.on_exception(backoff.expo, requests.exceptions.RequestException, max_tries=8)
def search_places(api_key, location, radius, keyword, next_page_token=None):
    url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    params = {
        'key': api_key,
        'location': location,
        'radius': radius,
        'keyword': keyword
    }
    if next_page_token:
        params['pagetoken'] = next_page_token

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        results = response.json()
        return results
    except requests.exceptions.RequestException as e:
        print(f"Error en la solicitud: {e}")
        return None

# funcion para obtener detalles adicionales de cada lugar
@backoff.on_exception(backoff.expo, requests.exceptions.RequestException, max_tries=8)
def get_place_details(api_key, place_id):
    url = 'https://maps.googleapis.com/maps/api/place/details/json'
    params = {
        'key': api_key,
        'place_id': place_id,
        'fields': 'name,formatted_address,geometry,formatted_phone_number,opening_hours,website,rating,reviews,price_level,user_ratings_total'
    }
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        details = response.json()
        return details
    except requests.exceptions.RequestException as e:
        print(f"Error en la solicitud: {e}")
        return None

# Función principal para obtener los datos de los lugares
def get_places_data(api_key, lat_min, lat_max, lng_min, lng_max, num_points_lat, num_points_lng, radius, keyword):
    places_data = []

    #calculo el tamaño de paso necesario para generar la cuadrícula
    lat_step = (lat_max - lat_min) / num_points_lat
    lng_step = (lng_max - lng_min) / num_points_lng
    #Bucle para iterar la N cantidad de puntos en la cuadricula predefinida
    for i in range(num_points_lat + 1):
        for j in range(num_points_lng + 1):
            lat = lat_min + i * lat_step
            lng = lng_min + j * lng_step
            location = f'{lat},{lng}'
            print(f'Buscando en {location}')

            next_page_token = None
            while True:
                results = search_places(api_key, location, radius, keyword, next_page_token)
                if results is None:
                    break
                for place in results.get('results', []):
                    place_id = place.get('place_id')
                    details = get_place_details(api_key, place_id)
                    if details and 'result' in details:
                        place_info = details['result']
                        place_data = {
                            'Nombre': place_info.get('name'),
                            'Dirección': place_info.get('formatted_address'),
                            'Latitud': place_info['geometry']['location'].get('lat'),
                            'Longitud': place_info['geometry']['location'].get('lng'),
                            'Teléfono': place_info.get('formatted_phone_number'),
                            'Horario': place_info.get('opening_hours', {}).get('weekday_text'),
                            'Página Web': place_info.get('website'),
                            'Rating': place_info.get('rating'),
                            'Reseñas': [review.get('text') for review in place_info.get('reviews', [])],
                            'Precio': place_info.get('price_level'),
                            'Popularidad': place_info.get('user_ratings_total')
                        }
                        places_data.append(place_data)

                next_page_token = results.get('next_page_token')
                if not next_page_token:
                    break

                time.sleep(2)  # Tiempor de espera para  los límites de la API

    return places_data

# una vez definida la funcion se ejecuta la función y obtengo los datos
places_data = get_places_data(api_key, lat_min, lat_max, lng_min, lng_max, num_points_lat, num_points_lng, radius, keyword)

# creamos un df con los resultados
df = pd.DataFrame(places_data)

# exportamos el df en un archivo csv
df.to_csv('tiendas_de_cortinas_chile.csv', index=False)

print("Datos guardados en 'tiendas_de_cortinas_chile.csv'")


Buscando en -56.0,-75.0
Buscando en -56.0,-74.55
Buscando en -56.0,-74.1
Buscando en -56.0,-73.65
Buscando en -56.0,-73.2
Buscando en -56.0,-72.75
Buscando en -56.0,-72.3
Buscando en -56.0,-71.85
Buscando en -56.0,-71.4
Buscando en -56.0,-70.95
Buscando en -56.0,-70.5
Buscando en -56.0,-70.05
Buscando en -56.0,-69.6
Buscando en -56.0,-69.15
Buscando en -56.0,-68.7
Buscando en -56.0,-68.25
Buscando en -56.0,-67.8
Buscando en -56.0,-67.35
Buscando en -56.0,-66.9
Buscando en -56.0,-66.45
Buscando en -56.0,-66.0
Buscando en -54.075,-75.0
Buscando en -54.075,-74.55
Buscando en -54.075,-74.1
Buscando en -54.075,-73.65
Buscando en -54.075,-73.2
Buscando en -54.075,-72.75
Buscando en -54.075,-72.3
Buscando en -54.075,-71.85
Buscando en -54.075,-71.4
Buscando en -54.075,-70.95
Buscando en -54.075,-70.5
Buscando en -54.075,-70.05
Buscando en -54.075,-69.6
Buscando en -54.075,-69.15
Buscando en -54.075,-68.7
Buscando en -54.075,-68.25
Buscando en -54.075,-67.8
Buscando en -54.075,-67.35
Buscando 

In [6]:
df

,Nombre,Dirección,Latitud,Longitud,Teléfono,Horario,Página Web,Rating,Reseñas,Precio,Popularidad
0,Blanqueria del Sur,"Héroes de Malvinas 14, Río Grande, Tierra del ...",-53.811413,-67.664900,None,"[Monday: 10:00 AM – 8:00 PM, Tuesday: 10:00 AM...",https://www.facebook.com/BlanqueriaDelSurr,4.8,"[Very good service, very affordable prices, I ...",None,4.0
1,Signo Sur letreros,"Juan Jose Paso 1285, V9420 Río Grande, Tierra ...",-53.805780,-67.747044,02964 48-9188,"[Monday: 9:00 AM – 5:00 PM, Tuesday: 9:00 AM –...",None,4.4,"[Very good attention and service., Great welco...",None,48.0
2,Blanqueria del Sur,"Héroes de Malvinas 14, Río Grande, Tierra del ...",-53.811413,-67.664900,None,"[Monday: 10:00 AM – 8:00 PM, Tuesday: 10:00 AM...",https://www.facebook.com/BlanqueriaDelSurr,4.8,"[Very good service, very affordable prices, I ...",None,4.0
3,Tienda del hogar,"Simón Bolívar 3475, V9420 Río Grande, Tierra d...",-53.802983,-67.750658,02964 61-2071,"[Monday: 11:00 AM – 9:00 PM, Tuesday: 11:00 AM...",None,NaN,[],None,NaN
4,Tapiceria Ariana,"Pje. Sta. Maria 860, V9420 Río Grande, Tierra ...",-53.815905,-67.669676,02964 55-3516,"[Monday: 10:00 AM – 8:00 PM, Tuesday: 10:00 AM...",https://m.facebook.com/tapiceria.ariana.9,5.0,[Very good service. Thank you very much Ariana...,None,4.0
...,...,...,...,...,...,...,...,...,...,...,...
966,Decore,"Av. América esquina, Cochabamba, Bolivia",-17.373135,-66.149308,None,"[Monday: 9:00 AM – 1:00 PM, 3:00 – 7:00 PM, Tu...",None,5.0,[],None,1.0
967,TAPICERIA “COVER TOÑO”,"Av. Guillermo Urquidi 1139, Cochabamba, Bolivia",-17.397943,-66.147280,61629004,"[Monday: 9:00 AM – 7:30 PM, Tuesday: 9:00 AM –...",None,5.0,"[Speed, efficiency and excellent work and price]",None,1.0
968,Decoraciones Hogar Entresueños Ltda.,"C. Lanza 367, Cochabamba, Bolivia",-17.388856,-66.154033,72269682,"[Monday: 9:00 AM – 7:00 PM, Tuesday: 9:00 AM –...",None,NaN,[],None,NaN
969,Sussy textiles,"Teofilo Vargas 407-351, Cochabamba, Bolivia",-17.375836,-66.164634,70783536,"[Monday: 9:00 AM – 12:00 PM, 2:30 – 6:00 PM, T...",https://www.facebook.com/TextilesSussy/,4.7,"[Nice choice of fabrics, very helpful and fri...",None,38.0


In [11]:
df.describe(include="all")

,Nombre,Dirección,Latitud,Longitud,Teléfono,Horario,Página Web,Rating,Reseñas,Precio,Popularidad
count,971,971,971.000000,971.000000,846,865,458,774.000000,971,0,774.000000
unique,657,671,NaN,NaN,543,429,258,NaN,467,0,NaN
top,Casaideas,"0000, Cochabamba, Bolivia",NaN,NaN,600 329 2002,"[Monday: Open 24 hours, Tuesday: Open 24 hours...",https://www.sodimac.cl/sodimac-cl,NaN,[],NaN,NaN
freq,12,6,NaN,NaN,17,28,15,NaN,197,NaN,NaN
mean,NaN,NaN,-35.050974,-70.385063,NaN,NaN,NaN,4.395866,NaN,NaN,263.947028
std,NaN,NaN,7.955219,2.419099,NaN,NaN,NaN,0.689026,NaN,NaN,1005.990047
min,NaN,NaN,-53.815995,-73.795853,NaN,NaN,NaN,1.000000,NaN,NaN,1.000000
25%,NaN,NaN,-38.767338,-72.590311,NaN,NaN,NaN,4.100000,NaN,NaN,3.000000
50%,NaN,NaN,-36.588922,-71.335423,NaN,NaN,NaN,4.500000,NaN,NaN,10.000000
75%,NaN,NaN,-32.897850,-68.144579,NaN,NaN,NaN,5.000000,NaN,NaN,57.000000


## **INTERPRETACION DE LAS COLUMNAS:**

1. **Nombre**: El nombre de la tienda o establecimiento encontrado.
   
2. **Direccion**: La dirección física donde se ubica la tienda.

3. **Latitud y Longitud**: Las coordenadas geográficas precisas de la ubicación de la tienda.

4. **Telefono**: El numero de teléfono de contacto de la tienda. 

5. **Horario**: Los horarios de apertura y cierre de la tienda durante los días de la semana.

6. **Pagina Web**: Enlaces a sitios web de la tienda, si están disponibles. Proporciona más información sobre la tienda y sus productos.

7. **Rating**: La calificación promedio de la tienda según los usuarios que la han evaluado.

8. **Reseñas**: Comentarios escritos por los usuarios que han visitado la tienda.

9. **Precio**: Esta columna no tiene valores.

10. **Popularidad**: Número total de calificaciones o reseñas recibidas por la tienda. Refleja la cantidad de interacción y feedback de los clientes.


## **CONCLUSIONES:**

Se llevó a cabo una búsqueda general para recopilar la información disponible. Considerar la reducción de la distancia entre los puntos de búsqueda podría resultar en un mayor número de registros. El correo electrónico no estaba incluido en la documentación de la API, por lo que no fue posible extraerlo como parámetro. Se observó que algunos datos corresponden a locales de Sodimac, lo cual podría influir en el análisis de la proximidad entre las tiendas en relación con estos locales. Es importante tener en cuenta que algunos puntos podrían estar ubicados fuera de Chile, ya que la consulta se realizó con coordenadas de 1 decimal, lo que reduce la precisión de los límites geográficos. Esto no necesariamente reduce la cantidad de datos, pero podría incluir información adicional que no pertenece a la región de interés.